In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

In [ ]:
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.retrievers import WikipediaRetriever

class DDGSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


class DDGSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    Use this tool to find a main keyword from query and get data from duckduckgo.
    It takes a query as an argument.
    Example query: Research about the XZ backdoor
    """
    args_schema: Type[DDGSearchToolArgsSchema] = DDGSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)
    

class WikipediaSearchToolArgsSchema(BaseModel):
    query : str = Field(description="")

class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Use this tool to find a main keyword from query and get data from wikipedia
    It takes a query as an argument.
    Example query: Research about the XZ backdoor
    """
    args_schema : Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query):
        retriever = WikipediaRetriever(top_k_results=2)
        docs = retriever.get_relevant_documents(query)
        docs = "\n\n".join(document.page_content for document in docs)
        return docs
    



    

In [ ]:
from langchain.agents.agent_toolkits import FileManagementToolkit


write_tool = FileManagementToolkit(
    selected_tools=["write_file"],
).get_tools()[0]


class MakeTextFileToolArgsSchema(BaseModel):
    file_name: str = Field("Name of a Text File")
    text_content: str = Field("Text content of a Text File")


class MakeTextFileTool(BaseTool):
    name = "MakeTextFile"
    description = """
    Use this to make .TXT file.
    You should enter a file_name and text_content.
    """
    args_schema: Type[MakeTextFileToolArgsSchema] = MakeTextFileToolArgsSchema

    def _run(self, file_name, text_content):
        write_tool.invoke(
            {
                "file_path": f"{file_name}.txt",
                "text": text_content,
            }
        )
        return "I made a report file."

In [62]:
from langchain.agents import initialize_agent, AgentType
from langchain.schema import SystemMessage

investor_agent = initialize_agent(
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    verbose=True,
    tools=[
        DDGSearchTool(),
        WikipediaSearchTool(),
        MakeTextFileTool(),
    ],
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
                You are a very meticulous Data searcher.

                You want to find information from every source possible.

                You can find information from both DuckDuckGo and Wikipedia.

                After searching data from duckduckgo and wikipedia, You should make a report txt file.
            """
        )
    },
)


prompt = """
Research about the XZ backdoor
"""

investor_agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearchTool` with `{'query': 'XZ backdoor'}`


In February of this year, Tan issued commits for versions 5.6.0 and 5.6.1 of XZ Utils. The updates implemented the backdoor. In the following weeks, Tan or others appealed to developers of Ubuntu ... On March 28, 2024 a backdoor was identified in XZ Utils. This vulnerability, CVE-2024-3094 with a CVSS score of 10 is a result of a software supply chain compromise impacting versions 5.6.0 and 5.6.1 of XZ Utils. The U.S. Cybersecurity and Infrastructure Security Agency (CISA) has recommended organizations to downgrade to a previous non-compromised XZ Utils version. That operation matches the style of the XZ Utils backdoor far more than the cruder supply chain attacks of APT41 or Lazarus, by comparison. "It could very well be someone else," says Aitel. The version of XZ should be carefully checked, as versions 5.6.0 and 5.6.1 contain the backdoor. It is advised to revert to a previou

c:\Users\CWS\Documents\NomadCoder\python_gpt\env\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, the backdoor only targeted Debian- and RPM-based systems running on the x86-64 architecture. At the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been assigned a CVSS score of 10.0, the highest possible score.


== Background ==
Microsoft employee and PostgreSQL developer Andres Freund reported the backdoor after investigating a performance regression in Debian Sid. Freund noticed that SSH connections were generating an unexpectedly high amount of CPU us

{'input': '\nResearch about the XZ backdoor\n',
 'output': 'I have created a research report on the XZ backdoor. The report includes information gathered from both DuckDuckGo and Wikipedia searches. The report covers details about the backdoor, background information, mechanism, features, usage, development, and adoption of XZ Utils.\n\nIf you would like to access the report file, please let me know.'}